In [1]:
from hypersurface_tf import *
from generate_h import *
from biholoNN import *
import tensorflow as tf
import numpy as np

### Prepare the dataset:

In [2]:
z0, z1, z2, z3, z4 = sp.symbols('z0, z1, z2, z3, z4')
Z = [z0,z1,z2,z3,z4]
f = z0**5 + z1**5 + z2**5 + z3**5 + z4**5 + 0.5*z0*z1*z2*z3*z4
np.random.seed(123)
HS = Hypersurface(Z, f, 10000)
HS_test = Hypersurface(Z, f, 10000)

In [3]:
train_set = generate_dataset(HS)
test_set = generate_dataset(HS_test)

In [4]:
train_set = train_set.shuffle(500000).batch(1000)
test_set = test_set.shuffle(500000).batch(1000)

### Build the model:

In [24]:
class KahlerPotential(tf.keras.Model):

    def __init__(self):
        super(KahlerPotential, self).__init__()
        self.biholomorphic = Biholomorphic()
        #self.layer1 = WidthOneDense()
        #self.layer1 = Dense(25, 1, activation=None)
        self.layer1 = Dense(25,50, activation=tf.square)
        self.layer2 = Dense(50,100, activation=tf.square)
        self.layer3 = Dense(100,250, activation=tf.square)
        self.layer4 = Dense(250,500, activation=tf.square)
        self.layer5 = Dense(500,500, activation=tf.square)
        #self.layer6 = Dense(500,500, activation=tf.square)

        
    def call(self, inputs):
        x = self.biholomorphic(inputs)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        #x = self.layer6(x)
        x = tf.reduce_sum(x, 1)
        x = tf.math.log(x)
        return x

In [25]:
model = KahlerPotential()

In [28]:
model.save('experiments.yidi/biholo/3layers/saved_model/50_100_100')

INFO:tensorflow:Assets written to: experiments.yidi/biholo/3layers/saved_model/50_100_100/assets


In [33]:
model = tf.keras.models.load_model('experiments.yidi/biholo/3layers/saved_model/50_100_100', compile=False)

In [34]:
@tf.function
def volume_form(x, Omega_Omegabar, mass, restriction):

    kahler_metric = complex_hessian(tf.math.real(model(x)), x)
    volume_form = tf.math.real(tf.linalg.det(tf.matmul(restriction, tf.matmul(kahler_metric, restriction, adjoint_b=True))))
    weights = mass / tf.reduce_sum(mass)
    factor = tf.reduce_sum(weights * volume_form / Omega_Omegabar)
    #factor = tf.constant(35.1774, dtype=tf.complex64)
    return volume_form / factor

In [44]:
def cal_total_loss(dataset, loss_function):
    
    total_loss = 0
    total_mass = 0
    
    for step, (points, Omega_Omegabar, mass, restriction) in enumerate(dataset):
        omega = volume_form(points, Omega_Omegabar, mass, restriction)
        mass_sum = tf.reduce_sum(mass)
        total_loss += loss_function(Omega_Omegabar, omega, mass) * mass_sum
        total_mass += mass_sum
   
    total_loss = total_loss / total_mass
    
    return total_loss.numpy()

In [45]:
optimizer = tf.keras.optimizers.Adam()
epochs = 2000

for epoch in range(epochs):
    for step, (points, Omega_Omegabar, mass, restriction) in enumerate(train_set):
        with tf.GradientTape() as tape:
            
            omega = volume_form(points, Omega_Omegabar, mass, restriction)
            loss = weighted_MAPE(Omega_Omegabar, omega, mass)
            grads = tape.gradient(loss, model.trainable_weights)
            #print(grads)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        if step % 500 == 0:
            print("step %d: loss = %.4f" % (step, loss))

    test_loss = cal_total_loss(test_set, weighted_MAPE)
    print("test_loss:", test_loss)

step 0: loss = 0.0094
test_loss: 0.009486649
step 0: loss = 0.0092
test_loss: 0.008963314
step 0: loss = 0.0089
test_loss: 0.009117666
step 0: loss = 0.0091
test_loss: 0.008896006
step 0: loss = 0.0089
test_loss: 0.008997408
step 0: loss = 0.0089
test_loss: 0.009082072
step 0: loss = 0.0094
test_loss: 0.009204662
step 0: loss = 0.0098


KeyboardInterrupt: 

In [46]:
train_loss = cal_total_loss(train_set, weighted_MAPE)

In [58]:
def delta_sigma_square_train(y_true, y_pred, mass):
    weights = mass / K.sum(mass)
    return K.sum((K.abs(y_true - y_pred) / y_true - train_loss)**2 * weights)

def delta_sigma_square_test(y_true, y_pred, mass):
    weights = mass / K.sum(mass)
    return K.sum((K.abs(y_true - y_pred) / y_true - test_loss)**2 * weights)

In [61]:
delta_sigma_train = math.sqrt(cal_total_loss(train_set, delta_sigma_square_train) / HS.n_points)
delta_sigma_test = math.sqrt(cal_total_loss(test_set, delta_sigma_square_test) / HS.n_points)

In [63]:
print(delta_sigma_train)
print(delta_sigma_test)

3.731694332240655e-05
3.812126437801893e-05
